In [ ]:
!pip install ipywidgets

In [10]:
import json
import ipywidgets as widgets
from IPython.display import display, HTML
from queue import PriorityQueue  # Import the PriorityQueue class

class Node:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.g = float('inf')
        self.f = float('inf')
        self.parent = None
        self.type = 'empty'

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a.row - b.row) + abs(a.col - b.col)

def a_star(start, goal, grid):
    open_list = PriorityQueue()
    closed_list = set()
    start.g = 0
    start.f = heuristic(start, goal)
    open_list.put((start.f, start))

    while not open_list.empty():
        _, current = open_list.get()

        if current == goal:
            path = []
            while current:
                path.append((current.row, current.col))
                current = current.parent
            return path[::-1]

        closed_list.add(current)
        for neighbor in get_neighbors(current, grid):
            if neighbor in closed_list or neighbor.type == 'obstacle':
                continue

            tentative_g = current.g + 1
            if tentative_g < neighbor.g:
                neighbor.parent = current
                neighbor.g = tentative_g
                neighbor.f = neighbor.g + heuristic(neighbor, goal)
                open_list.put((neighbor.f, neighbor))
    return []

def get_neighbors(node, grid):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    neighbors = []
    for dr, dc in directions:
        r, c = node.row + dr, node.col + dc
        if 0 <= r < len(grid) and 0 <= c < len(grid[0]):
            neighbors.append(grid[r][c])
    return neighbors

rows, cols = 20, 20
grid = [[None for _ in range(cols)] for _ in range(rows)]
startNode = None
goalNode = None
obstacles = []

def create_grid():
    grid_ui = widgets.GridBox()
    grid_ui.layout.grid_template_columns = " ".join(["30px"] * cols)
    grid_ui.layout.grid_template_rows = " ".join(["30px"] * rows)

    for row in range(rows):
        for col in range(cols):
            cell = widgets.Button(description=" ", layout=widgets.Layout(width="30px", height="30px", border="1px solid #000"))  # Added border here
            cell.on_click(lambda btn, r=row, c=col: handle_cell_click(btn, r, c))
            grid[row][col] = cell
            grid_ui.children += (cell,)

    return grid_ui

def handle_cell_click(btn, r, c):
    global startNode, goalNode, obstacles
    if not startNode:
        startNode = (r, c)
        btn.style.button_color = 'blue'
    elif not goalNode:
        goalNode = (r, c)
        btn.style.button_color = 'green'
    else:
        obstacles.append((r, c))
        btn.style.button_color = 'gray'

def start_pathfinding():
    global startNode, goalNode, obstacles
    if not startNode or not goalNode:
        print("Please select both start and goal points!")
        return

    start_row, start_col = startNode
    goal_row, goal_col = goalNode

    grid_nodes = [[Node(r, c) for c in range(cols)] for r in range(rows)]
    for r, c in obstacles:
        grid_nodes[r][c].type = 'obstacle'

    start = grid_nodes[start_row][start_col]
    goal = grid_nodes[goal_row][goal_col]

    path = a_star(start, goal, grid_nodes)
    for (r, c) in path:
        grid[r][c].style.button_color = 'yellow'

def clear_grid():
    global startNode, goalNode, obstacles
    startNode = None
    goalNode = None
    obstacles = []
    for r in range(rows):
        for c in range(cols):
            grid[r][c].style.button_color = None  # Reset to no color

# Buttons with borders and side-by-side layout
start_button = widgets.Button(description="Start Pathfinding", layout=widgets.Layout(border="1px solid #000", width="200px"))
start_button.on_click(lambda x: start_pathfinding())

clear_button = widgets.Button(description="Clear Grid", layout=widgets.Layout(border="1px solid #000", width="200px"))
clear_button.on_click(lambda x: clear_grid())

# Arrange buttons side by side using HBox
button_box = widgets.HBox([start_button, clear_button])

# Display grid and buttons
display(create_grid())
display(button_box)


GridBox(children=(Button(description=' ', layout=Layout(border='1px solid #000', height='30px', width='30px'),…

Please select both start and goal points!
